In [1]:
%load_ext sql
%sql sqlite://

'Connected: @None'

# Aggregation

In SQL, we can perform aggregation over multiple rows using `aggregate functions` 

## Aggregate Functions

So far, all SQL expressions have referred to the values in a single row at a time. If we take a look at the structure of a select statement once again,

<img src = 'expression.png' width = 700/>

...each of the `[expression]` can refer to column name in the `[table]`. But in actuality, they are referring to the value of a particular row for that column. 

When we use an `[expression]` to filter the rows of the `[table]` (e.g. `latitude >=43` [expression] for some `cities` [table]). We are evaluating whether the value of the latitude column for a particular row is above some threshold; if yes, keep that row, otherwise remove that row. The `[expression]` itself is evaluated per row.

An aggregate function in the `[columns]` clause computes a value from a group of rows (e.g. summing together values for a particular column).

Here we create a new table `animals`, containing the number of `legs` and the `weight` in pounds.

In [3]:
%%sql
create table animals as
    select "dog" as kind, 4 as legs, 20 as weight union
    select "cat", 4, 10 union
    select "ferret", 4, 10 union
    select "parrot", 2, 6 union
    select "penguin", 2, 10 union
    select "t-rex", 2, 12000;

 * sqlite://
Done.


[]

In [4]:
%%sql
select * from animals;

 * sqlite://
Done.


kind,legs,weight
cat,4,10
dog,4,20
ferret,4,10
parrot,2,6
penguin,2,10
t-rex,2,12000


An example usage of aggregate functions is to find the most legs using the aggregate function `max`,

In [6]:
%%sql
select max(legs) from animals;

 * sqlite://
Done.


max(legs)
4


And here is an example of calculating the sum of weights of all animals,

In [8]:
%%sql
select sum(weight) from animals;

 * sqlite://
Done.


sum(weight)
12056


The argument that we passed to `max` can be a combination. 

In [9]:
%%sql
select max(legs - weight) + 5 from animals;

 * sqlite://
Done.


max(legs - weight) + 5
1


Above, SQL grabbed the highest result "subtracting legs by weight, then adding 5" of each row. It appears that the greatest result can be obtained by calculating the `"parrot"` row.  

It's also possible to combine multiple different aggregations. 

In [10]:
%%sql
select max(legs), min(weight) from animals;

 * sqlite://
Done.


max(legs),min(weight)
4,6


Notice that of all the `animals` that we have, there is no row with `4 legs` and `6 weight`. The aggregate functions are called independently. We can even do arithmetic function with these independent aggregate functions,

In [11]:
%%sql
select max(legs) - min(weight) from animals;

 * sqlite://
Done.


max(legs) - min(weight)
-2


We can also combine the aggregate functions with a `where` clause. The following is an example of using the aggregate functions while excluding `"t-rex"`,

In [12]:
%%sql
select min(legs), max(weight) from animals
where kind <> "t-rex";

 * sqlite://
Done.


min(legs),max(weight)
2,20


We can also calculate the average value using aggregate function, 

In [13]:
%%sql
select avg(legs) from animals;

 * sqlite://
Done.


avg(legs)
3.0


We can also count the number of rows in a table. The `count` aggregate function can take a column name as the argument, but the column name does not matter since they all have the same number of rows.

In [15]:
%%sql
select count(legs) from animals;

 * sqlite://
Done.


count(legs)
6


In [17]:
%%sql
select count(legs), count(weight), count(kind) from animals;

 * sqlite://
Done.


count(legs),count(weight),count(kind)
6,6,6


A more typical syntax is to use `*` as the argument of `count`,

In [19]:
%%sql
select count(*) from animals;

 * sqlite://
Done.


count(*)
6


If that's the case, why would we even bother to use a column name as the argument? Here's why,

there's a special keyword `distinct` that we can write before a column name. This will count the number of distinct values. For example, there are 2 different numbers for `legs`, `4` and `2`.

In [20]:
%%sql
select count(distinct legs) from animals;

 * sqlite://
Done.


count(distinct legs)
2


And there are 4 different numbers for `weight`, `20`, `10`, `6`, and `12000`.

In [21]:
%%sql
select count(distinct weight) from animals;

 * sqlite://
Done.


count(distinct weight)
4


The `distinct` keyword can also be applied to `sum` to sum up all the unique values,

In [22]:
%%sql
select sum(distinct weight) from animals;

 * sqlite://
Done.


sum(distinct weight)
12036


## Mixing Aggregate Functions and Single Values

An aggregate function also selects a row in the table, which may be meaningful.

For example, if we want to find the animal with the most weight, we can do the following,

In [23]:
%%sql
select max(weight), kind from animals;

 * sqlite://
Done.


max(weight),kind
12000,t-rex


Now if we decide to `min(kind)`, then the row with the least alphabetical order of `kind` is selected,

In [24]:
%%sql
select min(kind), kind from animals;

 * sqlite://
Done.


min(kind),kind
cat,cat


We don't always obtain a result that makes sense. For example, if we try the following,

In [29]:
%%sql
select max(legs), kind from animals;

 * sqlite://
Done.


max(legs),kind
4,cat


...SQL arbitrarily chosen a row among the 3 different options since all those 3 have the most legs. The following gives an arbitrary outcome as well,

In [31]:
%%sql
select avg(weight), kind from animals;

 * sqlite://
Done.


avg(weight),kind
2009.3333333333333,cat
